<a href="https://colab.research.google.com/github/rahulk98/Deep_learning_algorithms/blob/master/Deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import the libraries required
import keras
from keras.datasets import mnist
from keras.layers import Dense
from keras.models import Sequential
import time


#Splitting the data to test and train
(x_train, y_train), (x_test, y_test) = mnist.load_data()


#Convert the images to vectors by flattening them
image_vector_size = 28*28
x_train = x_train.reshape(x_train.shape[0], image_vector_size)
x_test = x_test.reshape(x_test.shape[0], image_vector_size)



#Convert the output to one-hot encoded vectors
num_classes = 10                                          
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


#Creating a sequential model 
model = Sequential()
#Adding the first hidden layer with 32 nodes. The input layer has 784 nodes
model.add(Dense(units=397, activation='sigmoid', kernel_initializer='uniform', input_shape=(image_vector_size,)))
#Adding the output layer
model.add(Dense(units=num_classes, activation='softmax'))
model.summary()


start = time.time()


#Training the model

model.compile(optimizer="sgd", loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=15, verbose=False, validation_split=.1)

end = time.time()

loss, accuracy  = model.evaluate(x_test, y_test, verbose=False)
print(accuracy)

time_taken = end - start                                                                #time taken
print('Time taken: ',time_taken)


Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 397)               311645    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                3980      
Total params: 315,625
Trainable params: 315,625
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
0.9548
Time taken:  43.91749858856201


In [2]:
import numpy as np

from sklearn import linear_model, datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.base import clone

import time
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split




# Turn down for faster convergence
train_samples = 60000

# Load data from https://www.openml.org/d/554
x, y = fetch_openml('mnist_784', version=1, return_X_y=True)


x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size=train_samples, test_size=10000)

##############################################################################################################




x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

x_train = x_train/255 # 0-1 scaling
x_test =  x_test/255
y_train = y_train
y_test = y_test


logistic = linear_model.LogisticRegression(solver='newton-cg', tol=1,
                                           multi_class='multinomial')
rbm = BernoulliRBM(random_state=0, verbose=True)

rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('logistic', logistic)])



# #############################################################################
# Training

rbm.learning_rate = 0.005
rbm.n_iter = 7
# More components tend to give better prediction performance, but larger
# fitting time
rbm.n_components = 100
logistic.C = 10.0

# Training RBM-Logistic Pipeline


rbm_features_classifier.fit(x_train, y_train)





# # #############################################################################
# # Evaluation

y_pred = rbm_features_classifier.predict(x_test) 
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.accuracy_score(y_test, y_pred)))









[BernoulliRBM] Iteration 1, pseudo-likelihood = -129.49, time = 14.08s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -111.77, time = 15.75s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -103.77, time = 15.78s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -98.12, time = 15.86s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -94.43, time = 15.74s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -91.48, time = 15.80s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -89.75, time = 15.73s
Logistic regression using RBM features:
0.9317



In [3]:
!pip install git+git://github.com/albertbup/deep-belief-network.git

  Cloning git://github.com/albertbup/deep-belief-network.git to /tmp/pip-req-build-u4yq4fhk
  Running command git clone -q git://github.com/albertbup/deep-belief-network.git /tmp/pip-req-build-u4yq4fhk
  Stored in directory: /tmp/pip-ephem-wheel-cache-7gjbw5cm/wheels/29/6d/3b/6a50cf42a32bdfaa903b17832d60d8d3e5dc4b0fd02ae8acaf
Successfully built deep-belief-network


In [4]:
import numpy as np


np.random.seed(1337)  # for reproducibility
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score

from dbn.tensorflow import SupervisedDBNClassification

# Loading dataset
digits = load_digits()
X, Y = digits.data, digits.target

# Data scaling
X = (X / 16).astype(np.float32)

# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[256, 256],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=100,
                                         batch_size=32,
                                         activation_function='relu',
                                         dropout_p=0.2)
classifier.fit(X_train, Y_train)

# Test
Y_pred = classifier.predict(X_test)
print('Done.\nAccuracy: %f' % accuracy_score(Y_test, Y_pred))

[START] Pre-training step:
Instructions for updating:
Use tf.cast instead.
>> Epoch 1 finished 	RBM Reconstruction error 3.221450
>> Epoch 2 finished 	RBM Reconstruction error 2.169739
>> Epoch 3 finished 	RBM Reconstruction error 1.828885
>> Epoch 4 finished 	RBM Reconstruction error 1.628937
>> Epoch 5 finished 	RBM Reconstruction error 1.407109
>> Epoch 6 finished 	RBM Reconstruction error 1.287342
>> Epoch 7 finished 	RBM Reconstruction error 1.186931
>> Epoch 8 finished 	RBM Reconstruction error 1.098507
>> Epoch 9 finished 	RBM Reconstruction error 1.038850
>> Epoch 10 finished 	RBM Reconstruction error 0.969662
>> Epoch 1 finished 	RBM Reconstruction error 2.863971
>> Epoch 2 finished 	RBM Reconstruction error 1.770083
>> Epoch 3 finished 	RBM Reconstruction error 1.227978
>> Epoch 4 finished 	RBM Reconstruction error 1.014690
>> Epoch 5 finished 	RBM Reconstruction error 0.984450
>> Epoch 6 finished 	RBM Reconstruction error 0.776238
>> Epoch 7 finished 	RBM Reconstruction erro